In [1]:
import os
import pandas as pd
from Bio import SeqIO

In [2]:
os.chdir("/fs/scratch/PAS0439/Ming/GutEuk_benchmark")

In [3]:
seqorigin = pd.read_csv("/fs/ess/PAS0439/MING/cilates_fungi_classifier/testset_seq_origin_chopped.csv")

In [4]:
tiara_proka = pd.read_csv("proka_tiara.txt", sep = "\t")
tiara_fungi = pd.read_csv("fungi_tiara.txt", sep = "\t")
tiara_protozoa = pd.read_csv("protozoa_tiara.txt", sep = "\t")
tiara_sags = pd.read_csv("sags_tiara.txt", sep = "\t")

/tmp/slurmtmp.25904731/ipykernel_48016/748316828.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  tiara_proka = pd.read_csv("proka_tiara.txt", sep = "\t")
/tmp/slurmtmp.25904731/ipykernel_48016/748316828.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  tiara_fungi = pd.read_csv("fungi_tiara.txt", sep = "\t")


In [5]:
tiara_fungi["sequence_id"] = [f[0] for f in tiara_fungi.sequence_id.str.split(" ")]
tiara_fungi["category"] = "fungi"
tiara_proka["sequence_id"] = [f[0] for f in tiara_proka.sequence_id.str.split(" ")]
tiara_proka["category"] = "prokaryotes"
tiara_protozoa["sequence_id"] = [f[0] for f in tiara_protozoa.sequence_id.str.split(" ")]
tiara_protozoa["category"] = "protozoa"
tiara_sags["sequence_id"] = [f[0] for f in tiara_sags.sequence_id.str.split(" ")]
tiara_sags["category"] = "protozoa"

In [6]:
tiara = pd.concat([tiara_fungi, tiara_proka, tiara_protozoa, tiara_sags])

In [7]:
tiara.rename(columns = {"class_fst_stage":"prediction"}, inplace = True)
tiara.drop("class_snd_stage", axis = 1, inplace = True)

In [8]:
def new_cate(x):
    if x in ['archaea', 'bacteria', 'prokarya']:
        x = "prokaryotes"
    elif x == "eukarya":
        x = "eukaryotes"
    else:
        x = "undetermined"
    return x
        
tiara.prediction = tiara.prediction.apply(new_cate)

In [9]:
prediction = []
category = []
sequence_id = []
records = SeqIO.parse("fungi_eukrep_euk.fa", 'fasta')
for record in records:
    sequence_id.append(str(record.id))
    category.append("fungi")
    prediction.append("eukaryotes")
    
records = SeqIO.parse("fungi_eukrep_pro.fa", 'fasta')
for record in records:
    sequence_id.append(str(record.id))
    category.append("fungi")
    prediction.append("prokaryotes")
    

records = SeqIO.parse("protozoa_eukrep_euk.fa", 'fasta')
for record in records:
    sequence_id.append(str(record.id))
    category.append("protozoa")
    prediction.append("eukaryotes")
    
records = SeqIO.parse("protozoa_eukrep_pro.fa", 'fasta')
for record in records:
    sequence_id.append(str(record.id))
    category.append("protozoa")
    prediction.append("prokaryotes")
    

records = SeqIO.parse("proka_eukrep_euk.fa", 'fasta')
for record in records:
    sequence_id.append(str(record.id))
    category.append("prokaryotes")
    prediction.append("eukaryotes")
    
records = SeqIO.parse("proka_eukrep_pro.fa", 'fasta')
for record in records:
    sequence_id.append(str(record.id))
    category.append("prokaryotes")
    prediction.append("prokaryotes")
    

records = SeqIO.parse("sags_eukrep_euk.fa", 'fasta')
for record in records:
    sequence_id.append(str(record.id))
    category.append("protozoa")
    prediction.append("eukaryotes")
    
records = SeqIO.parse("sags_eukrep_pro.fa", 'fasta')
for record in records:
    sequence_id.append(str(record.id))
    category.append("protozoa")
    prediction.append("prokaryotes")

In [10]:
eukrep = pd.DataFrame.from_dict({"category":category, "prediction":prediction, "sequence_id": sequence_id  })

In [11]:
eukrep.to_csv("/fs/ess/PAS0439/MING/cilates_fungi_classifier/stats_visualization/eukrep_testset_chopped.csv", index = None)

In [12]:
tiara.to_csv("/fs/ess/PAS0439/MING/cilates_fungi_classifier/stats_visualization/tiara_testset_chopped.csv", index = None)

In [13]:
eukrep.rename(columns = {"prediction":"eukrep"}, inplace = True)

In [14]:
tiara.rename(columns = {"prediction":"tiara"}, inplace = True)

In [15]:
guteuk = pd.read_csv("/fs/ess/PAS0439/MING/cilates_fungi_classifier/stats_visualization/GutEuk_benchmark_full.csv")

In [16]:
tmp = pd.merge(guteuk, tiara, left_on="sequence", right_on = "sequence_id")

In [17]:
tiara_benchmark_by_length = pd.merge(tmp, eukrep, left_on="sequence", right_on = "sequence_id")

In [18]:
tiara_benchmark_by_length.loc[tiara_benchmark_by_length.query('genome_type != "prokaryotes"').index, "genome_type"] = "eukaryotes"

In [19]:
def ceiling(x:int):
    x = (x // 5000)*5000
    return x
    
tiara_benchmark_by_length.loc[:,"sequence_length_range"] = tiara_benchmark_by_length.seq_length.apply(ceiling)

In [20]:
# sequence_length_range: 0 (3000 < x < 5000)
precision = []
recall = []
sequence_length = []
category = []
tool = []
for cate in ["prokaryotes", "eukaryotes"]:
    for seq_len in [0, 5000, 10000, 15000, 20000, 25000, 30000, 35000, 40000, 45000, 50000]:
        tp =  len(tiara_benchmark_by_length.query('genome_type == @cate').query("sequence_length_range == @seq_len").query('tiara == @cate'))
        fp = len(tiara_benchmark_by_length.query('genome_type != @cate').query("sequence_length_range == @seq_len").query('tiara == @cate'))
        fn = len(tiara_benchmark_by_length.query('genome_type == @cate').query("sequence_length_range == @seq_len").query('tiara != @cate'))
        if fn + fp + tp == 0:
            continue
        else:
            preci = tp/ (tp + fp)
            recal = tp/ (tp + fn)
            category.append(cate)
            sequence_length.append(seq_len)
            precision.append(preci)
            recall.append(recal)
            tool.append("Tiara")
        

for cate in ["prokaryotes", "eukaryotes"]:
    for seq_len in [0, 5000, 10000, 15000, 20000, 25000, 30000, 35000, 40000, 45000, 50000]:
        tp =  len(tiara_benchmark_by_length.query('genome_type == @cate').query("sequence_length_range == @seq_len").query('eukrep == @cate'))
        fp = len(tiara_benchmark_by_length.query('genome_type != @cate').query("sequence_length_range == @seq_len").query('eukrep == @cate'))
        fn = len(tiara_benchmark_by_length.query('genome_type == @cate').query("sequence_length_range == @seq_len").query('eukrep != @cate'))
        if fn + fp + tp == 0:
            continue
        else:
            preci = tp/ (tp + fp)
            recal = tp/ (tp + fn)
            category.append(cate)
            sequence_length.append(seq_len)
            precision.append(preci)
            recall.append(recal)
            tool.append("EukRep")
        

In [21]:
tiara_eukrep_preci_recall_by_len = pd.DataFrame.from_dict({"category":category, "sequence_length":sequence_length, "precision":precision, "recall":recall, "tool": tool})
tiara_eukrep_preci_recall_by_len.to_csv("/fs/ess/PAS0439/MING/cilates_fungi_classifier/stats_visualization/tiara_eukrep_preci_recall_by_len.csv", index = None)